#### PDF RAG - 소비자 분쟁조정 사례집 QA

## PDF Data

In [2]:
%pip install -U -q langchain langchain-core langchain_community langchain-ollama langchain_experimental pypdf pydantic

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from glob import glob

pdf_files = glob(os.path.join('data', '*분쟁*.pdf'))
len(pdf_files) 
print(pdf_files)

['data\\2018서비스집단분쟁조성사례집.pdf']


In [4]:
# pdf 파일 목록
PDF_PATH = pdf_files[0]
print(PDF_PATH)

data\2018서비스집단분쟁조성사례집.pdf


In [5]:
# pdf 파일을 읽어서 텍스트로 변환
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_PATH)
data = loader.load()

print(type(data), len(data))

<class 'list'> 200


In [6]:
# 첫번째 페이지의 텍스트 출력
from pprint import pprint

pprint(data[0].page_content) 

'소비자분쟁조정위원회\n2018\n서비스·집단 \n분쟁조정 사례집'


In [7]:
# 첫번째 페이지의 메타데이터 출력
pprint(data[0].metadata)  

{'author': 'PC_A2',
 'creationdate': '2019-06-05T11:33:24+09:00',
 'creator': 'PScript5.dll Version 5.2.2',
 'moddate': '2019-06-05T11:58:31+09:00',
 'page': 0,
 'page_label': '1',
 'producer': 'Acrobat Distiller 9.0.0 (Windows)',
 'source': 'data\\2018서비스집단분쟁조성사례집.pdf',
 'title': '<32303139303630355FBCD2BAF1C0DABFF820BBE7B7CAC1FD5BBCADBAF1BDBA5D5FB3BBC1F65FC6EDC1FDBABB76657231312E687770>',
 'total_pages': 200}


In [8]:
# 10번째 페이지의 텍스트 출력
print(type(data[10].page_content))
pprint(data[10].page_content)

<class 'str'>
('제1장\n'
 '일\n'
 '반\n'
 '분\n'
 '쟁\n'
 '조\n'
 '정\n'
 ' 사\n'
 '례(\n'
 '서\n'
 '비\n'
 '스)\n'
 '제1장 일반분쟁조정 사례(서비스) ● 3\n'
 '사\n'
 '례 01 사건번호 2018일나565  | 결정일자 2018. 8. 7.\n'
 '세탁 후 갑피 마모 및 경화된 가죽 \n'
 '운동화에 대한 손해배상 요구\n'
 '주 문\n'
 '1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n'
 '운동화, 색상 : 흰색) 1켤레를 반환한다. \n'
 '2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n'
 '을 지급한다.\n'
 '이 유\n'
 '1. 기초사실\n'
 '가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n'
 '이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. \n'
 '피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n'
 '마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n'
 '으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신\n'
 '청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없\n'
 '다는 이유로 이를 거부하였다.\n'
 '나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n'
 '   \n'
 ' 신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n'
 '으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n'
 '인지 단정하기 어려운바, 판단 불가하다.')


정규식 설명

* 사\s*\n?\s*례\s*\d+  => "사례" 사이에 개행이 있을 수도 있으므로 유연하게 처리
* \s*사건번호\s*\d+ => "사건번호 2018일나565" 형식 매칭
* .*?| => 파이프(`
* \s*결정일자\s*\d{4}\.\s?\d{1,2}\.\s?\d{1,2}\. => 결정일자 "YYYY. M. D." 매칭

In [9]:
# 10번째 페이지의 텍스트에서 정보 추출
import re

split_pattern = r'사\s*\n?\s*례\s*\d+\s*사건번호\s*\d+.*?\|\s*결정일자\s*\d{4}\.\s?\d{1,2}\.\s?\d{1,2}\.'

split_text = re.findall(split_pattern, "".join(data[10].page_content), flags=re.DOTALL)
split_text

['사\n례 01 사건번호 2018일나565  | 결정일자 2018. 8. 7.']

In [10]:
# split_text가 존재하면 split_pattern으로 분리
if split_text:
    parts = re.split(split_pattern, "".join(data[10].page_content))

print(type(parts), len(parts))

<class 'list'> 2


In [11]:
# 분리된 텍스트 출력
parts[0]

'제1장\n일\n반\n분\n쟁\n조\n정\n 사\n례(\n서\n비\n스)\n제1장 일반분쟁조정 사례(서비스) ● 3\n'

In [12]:
# 분리된 텍스트 출력
parts[1]

'\n세탁 후 갑피 마모 및 경화된 가죽 \n운동화에 대한 손해배상 요구\n주 문\n1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n운동화, 색상 : 흰색) 1켤레를 반환한다. \n2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n을 지급한다.\n이 유\n1. 기초사실\n가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. \n피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신\n청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없\n다는 이유로 이를 거부하였다.\n나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n   \n 신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n인지 단정하기 어려운바, 판단 불가하다.'

In [13]:
# 특정 문자열의 위치 찾기

print(type(re.search(r'주 문\n', parts[1]).span()), re.search(r'주 문\n', parts[1]).span())
print(re.search(r'주 문\n', parts[1]).span()[0])

<class 'tuple'> (38, 42)
38


In [14]:

# [:38]를 사용하여 "주 문\n" 이전 부분만 추출하게 됨.
str_index = re.search(r'주 문\n', parts[1]).span()[0]
title = parts[1][:str_index].strip()
title

'세탁 후 갑피 마모 및 경화된 가죽 \n운동화에 대한 손해배상 요구'

In [15]:
# 내용 추출 "주 문\n" 이후 부분만 추출
content = parts[1][str_index:]
content

'주 문\n1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n운동화, 색상 : 흰색) 1켤레를 반환한다. \n2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n을 지급한다.\n이 유\n1. 기초사실\n가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. \n피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신\n청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없\n다는 이유로 이를 거부하였다.\n나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n   \n 신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n인지 단정하기 어려운바, 판단 불가하다.'

In [16]:
# 정규식으로 추출한 구분 문자열에서 사례 번호 추출
split_text[0]

'사\n례 01 사건번호 2018일나565  | 결정일자 2018. 8. 7.'

In [17]:
re.findall(r'례\s?(\d+)\s?사건번호', split_text[0])[0]

'01'

In [18]:
# 사건에 대한 메타데이터 추출하는 Pydantic 스키마 정의
from pydantic import BaseModel,Field

# Pydantic 모델 수정 (required 제거)
class Case(BaseModel):
    case_number: str = Field(..., description="The number of the case")
    case_date: str = Field(..., description="The date when the case occurred (year, month, day)")    

In [21]:
# Extraction chain 구성
from langchain_core.prompts import PromptTemplate
from langchain_experimental.llms.ollama_functions import OllamaFunctions
#from langchain_ollama import ChatOllama

# 템플릿 정의
prompt = PromptTemplate.from_template(
    """Extract the following fields from the given text and return a JSON object **without any extra nesting**.

TEXT: {text}

Return a JSON object with the following structure:

```json
{{
    "case_number": "2018일나565",
    "case_date": "2018. 8. 7."
}}"""
)

# 처리할   텍스트
text = split_text[0]

# OllamaFunctions 초기화
llm = OllamaFunctions(model="llama3", format="json", temperature=0)
#llm = ChatOllama(model="llama3", format="json", temperature=0)
runnable = prompt | llm.with_structured_output(schema=Case)

# 템플릿 실행
raw_response  = runnable.invoke({"text": text})

# 응답 처리
if "properties" in raw_response: 
    clean_response = raw_response["properties"] 
else: 
    clean_response = raw_response

# clean_response가 이미 Case 객체인지 확인
if isinstance(clean_response, Case):
    parsed_response = clean_response
else:
    parsed_response = Case(**clean_response)

print(parsed_response)



case_number='2018일나565' case_date='2018. 8. 7.'


In [22]:
dict(parsed_response)

{'case_number': '2018일나565', 'case_date': '2018. 8. 7.'}

In [23]:
data[-2].page_content

'2018 서비스·집단 분쟁조정사례집\n인 쇄 / 2019년 5월\n발 행 / 2019년 5월\n발행인 / 한국소비자원 원장 이 희 숙\n편집인 / 소비자분쟁조정위원회 위원장 \n제 작 / 소비자분쟁조정위원회 분쟁조정사무국\n디자인․인쇄 / (사)아름다운사람들복지회\n발행처 / 한국소비자원 소비자분쟁조정위원회\n\U000f02b2\U000f02b7\U000f02b7\U000f02b3\U000f02b8\n충청북도 음성군 맹동면 용두로 54\n전 화 / 043-880-5500'

In [24]:
# 문서 객체를 페이지별로 순회하며 사례 번호 등 메타데이터를 추출하고 업데이트
pdf_docs = []
case_metadata = {}
split_pattern = r'사\s*\n?\s*례\s*\d+\s*사건번호\s*\d+.*?\|\s*결정일자\s*\d{4}\.\s?\d{1,2}\.\s?\d{1,2}\.'

for doc in data[10:-2]:
    split_text = re.findall(split_pattern, "".join(doc.page_content))
    if split_text:

        # case id
        case_metadata['case_id'] = re.findall(r'례\s?(\d+)\s?사건번호', split_text[0])[0]

        parts = re.split(split_pattern, "".join(doc.page_content))

        if re.search(r'주 문\n', parts[1]):
            str_index = re.search(r'주 문\n', parts[1]).span()[0]
            # Add title to metadata
            case_metadata['title'] = parts[1][:str_index].replace('\n', '').strip()
                    
            # Update content
            doc.page_content = parts[1][str_index:].strip()
        else:
            case_metadata['title'] = ''
            
        # Extract metadata from text
        i = 0
        while i < 10:
            try:
                response = runnable.invoke({"text": split_text[0]})
                for k, v in dict(response).items():
                    case_metadata[k] = v.replace("\n", "").replace(" ", "")
                break
            except:
                i += 1
                continue

        # Update metadata 
        doc.metadata.update(case_metadata)

        # Append to split_docs
        pdf_docs.append(doc)

    else:
        # Update metadata 
        doc.metadata.update(case_metadata) 

        # Append to split_docs
        pdf_docs.append(doc)

len(pdf_docs)

188

In [25]:
pprint(pdf_docs[0].page_content)

('주 문\n'
 '1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n'
 '운동화, 색상 : 흰색) 1켤레를 반환한다. \n'
 '2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n'
 '을 지급한다.\n'
 '이 유\n'
 '1. 기초사실\n'
 '가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n'
 '이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. \n'
 '피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n'
 '마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n'
 '으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신\n'
 '청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없\n'
 '다는 이유로 이를 거부하였다.\n'
 '나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n'
 '   \n'
 ' 신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n'
 '으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n'
 '인지 단정하기 어려운바, 판단 불가하다.')


In [26]:
pprint(pdf_docs[0].metadata)

{'author': 'PC_A2',
 'case_date': '2018.8.7.',
 'case_id': '01',
 'case_number': '2018일나565',
 'creationdate': '2019-06-05T11:33:24+09:00',
 'creator': 'PScript5.dll Version 5.2.2',
 'moddate': '2019-06-05T11:58:31+09:00',
 'page': 10,
 'page_label': '11',
 'producer': 'Acrobat Distiller 9.0.0 (Windows)',
 'source': 'data\\2018서비스집단분쟁조성사례집.pdf',
 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구',
 'total_pages': 200}


## Embedding

In [27]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

C:\Users\vega2\AppData\Local\Temp\ipykernel_26764\1172022402.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\vega2\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vega2\.cache\huggingface\hub\models--snunlp--KR-SBERT-V40K-klueNLI-augSTS. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
# 문서를 임베딩
embeddings = embeddings_model.embed_documents(
    [
        "온라인 쇼핑몰에서 주문한 제품이 불량품으로 배송되었습니다. 이에 대한 법적 책임은 누구에게 있나요?",
        "구입한 전자제품이 고장나서 환불을 요청했지만 거부당했습니다. 피해 보상을 받을 수 있나요?",
        "호텔 예약 후 도착했는데 예약이 취소되었다고 했습니다. 이에 대한 대응 방법은 무엇인가요?",
        "자동차 수리 후 동일한 문제가 재발했습니다. 수리업체에 대한 법적 조치를 취할 수 있나요?",
        "항공편이 지연되어 중요한 일정을 놓쳤습니다. 이에 대한 피해 보상을 받을 수 있나요?"
    ]
)
print(len(embeddings), len(embeddings[0]))

5 768


In [29]:
embedded_query = embeddings_model.embed_query("에어컨 제품 불량에 대해서 보상을 받을 수 있을까요?")
print(embedded_query[:5])

[0.7448060512542725, -0.2612636089324951, -0.2569149136543274, 0.12548893690109253, -0.025863895192742348]


In [30]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 쿼리와 문서 간의 코사인 유사도 계산
for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

0.4530433066559124
0.6305271688030667
0.38549627935161485
0.4729425910947917
0.3779668033493392


## Chunking

In [31]:
# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

text = "에어컨 제품 불량에 대해서 보상을 받을 수 있을까요?"
encoded = tokenizer.encode(text)
print(len(text), len(encoded))
print(encoded)

29 12
[2, 19224, 8918, 20035, 5051, 9665, 22898, 9622, 3317, 21981, 35, 3]


In [32]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 120,
    chunk_overlap  = 10,
)

split_docs = text_splitter.split_documents(pdf_docs)
print(len(split_docs))
print(split_docs[0])

929
page_content='주 문
1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 
운동화, 색상 : 흰색) 1켤레를 반환한다. 
2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원
을 지급한다.
이 유
1. 기초사실' metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2019-06-05T11:33:24+09:00', 'author': 'PC_A2', 'moddate': '2019-06-05T11:58:31+09:00', 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구', 'source': 'data\\2018서비스집단분쟁조성사례집.pdf', 'total_pages': 200, 'page': 10, 'page_label': '11', 'case_id': '01', 'case_number': '2018일나565', 'case_date': '2018.8.7.'}


In [33]:
sample_text = split_docs[0].page_content
sample_encoded = tokenizer.encode(sample_text)
len(sample_text), len(sample_encoded)

(152, 71)

In [34]:
print(len(tokenizer.encode(split_docs[0].page_content)))
pprint(split_docs[0].page_content)

71
('주 문\n'
 '1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n'
 '운동화, 색상 : 흰색) 1켤레를 반환한다. \n'
 '2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n'
 '을 지급한다.\n'
 '이 유\n'
 '1. 기초사실')


In [35]:
print(len(tokenizer.encode(split_docs[1].page_content)))
pprint(split_docs[1].page_content)

83
('1. 기초사실\n'
 '가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n'
 '이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. \n'
 '피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피')


In [36]:
# 마침표 뒤에 나오는 줄바꿈 문자는 그대로 두고 나머지 줄바꿈 문자만 제거
result = re.sub(r'(?<!\.)\n', ' ', split_docs[1].page_content)
print(result)

1. 기초사실 가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색,  이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10.  피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피


In [37]:
pprint(split_docs[0].metadata)

{'author': 'PC_A2',
 'case_date': '2018.8.7.',
 'case_id': '01',
 'case_number': '2018일나565',
 'creationdate': '2019-06-05T11:33:24+09:00',
 'creator': 'PScript5.dll Version 5.2.2',
 'moddate': '2019-06-05T11:58:31+09:00',
 'page': 10,
 'page_label': '11',
 'producer': 'Acrobat Distiller 9.0.0 (Windows)',
 'source': 'data\\2018서비스집단분쟁조성사례집.pdf',
 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구',
 'total_pages': 200}


In [38]:
pprint(split_docs[1].metadata)

{'author': 'PC_A2',
 'case_date': '2018.8.7.',
 'case_id': '01',
 'case_number': '2018일나565',
 'creationdate': '2019-06-05T11:33:24+09:00',
 'creator': 'PScript5.dll Version 5.2.2',
 'moddate': '2019-06-05T11:58:31+09:00',
 'page': 10,
 'page_label': '11',
 'producer': 'Acrobat Distiller 9.0.0 (Windows)',
 'source': 'data\\2018서비스집단분쟁조성사례집.pdf',
 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구',
 'total_pages': 200}


In [39]:
f"### 이 사건은 '{split_docs[1].metadata['title']}'에 대한 사례입니다."

"### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다."

# Indexing

In [40]:
final_docs = []
for doc in split_docs:
    doc.page_content = f"### 이 사건은 '{doc.metadata['title']}'에 대한 사례입니다.\n\n" + \
    re.sub(r'(?<!\.)\n', ' ', doc.page_content)
    final_docs.append(doc)

print(final_docs[0].page_content)

### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

주 문 1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽  운동화, 색상 : 흰색) 1켤레를 반환한다.  2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원 을 지급한다.
이 유 1. 기초사실


In [41]:
print(final_docs[1].page_content)

### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

1. 기초사실 가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색,  이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10.  피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피


In [42]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=final_docs, 
                                    embedding=embeddings_model, 
                                    collection_name="consumer_case_qa",
                                    persist_directory="./db/chroma_db")

In [43]:
chroma_docs = vectorstore.similarity_search("세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?", k=5)
for doc in chroma_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:200])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

책임있는 사유로 세탁물이 손상, 색상변화, 얼룩 등의 하자가 발생였을 때에는 해당  세탁물에 대하여 세탁업자는 고객에게 세탁요금을 청구하지 못하므로, 세탁업자인 피 신청인이 세탁비 4,000원을 신청인에게 환급이 상당하다.
이상을 종합하면, 신청인은 
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여 피신청인은 이 사건 제품을 인수하였을 당시 이미 제품 상태가 좋지 않았 고, 한국소비자원의 신발제품심의위원회에서도 세탁과실로 인정하지 않았기 때문에 신 청인의 요구를 수용할 수 
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였 으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신 청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없 다는 이유
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

살피건대, 피신청인은 세탁 전부터 이 사건 제품의 상태가 좋지 않았다고 주장하나,  다음과 같은 사정들, 즉 ① 피신청인이 세탁을 위해 이 사건 제품을 인수하면서 세탁 물의 하자유무가 작성된 인수증을 교부하지 않은 점, ② 「세탁업 표준약관」 제3조 
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만 으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제 인지 단정하기 어려운바, 판단 불가하

## Retrieval

In [44]:
# Top K
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

책임있는 사유로 세탁물이 손상, 색상변화, 얼룩 등의 하자가 발생였을 때
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

살피건대, 피신청인은 세탁 전부터 이 사건 제품의 상태가 좋지 않았다고 
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상


In [45]:
# metadata를 이용한 필터링

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 5,
        'filter': {'case_id':'01'}
        }
)


query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

책임있는 사유로 세탁물이 손상, 색상변화, 얼룩 등의 하자가 발생였을 때
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

살피건대, 피신청인은 세탁 전부터 이 사건 제품의 상태가 좋지 않았다고 
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상


In [46]:
# page_content를 이용한 필터링

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 5,
        'where_document': {'$contains': '세탁'}
        }
)

query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

책임있는 사유로 세탁물이 손상, 색상변화, 얼룩 등의 하자가 발생였을 때
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

살피건대, 피신청인은 세탁 전부터 이 사건 제품의 상태가 좋지 않았다고 
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상


## Generation

In [49]:

from langchain_ollama import ChatOllama
#from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [50]:
# RAG Chain
llm = ChatOllama(model="qwen2", temperature=0)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

item = "세탁"
query = f"{item} 불량에 대한 손해배상은 어떻게 이루어지나요?"

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 2,
        'where_document': {'$contains': item}
        }
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


response = rag_chain.invoke(query)
response

'세탁 불량에 대한 손해배상은 다음과 같이 이루어집니다:\n\n1. 신청인은 피신청인에게 이 사건 제품을 반환해야 합니다.\n2. 피신청인은 손해배상액 67,000원(112,140원의 60%에서 1,000원 미만 버림)를 신청인에게 지불해야 합니다.\n3. 또한 피신청인은 세탁비 4,000원을 환급해야 합니다.\n\n따라서, 이 사건 손해배상과 세탁비 환급은 신청인의 제품 반환과 피신청인의 금액 지불로 이루어집니다.'

In [51]:
print(response)

세탁 불량에 대한 손해배상은 다음과 같이 이루어집니다:

1. 신청인은 피신청인에게 이 사건 제품을 반환해야 합니다.
2. 피신청인은 손해배상액 67,000원(112,140원의 60%에서 1,000원 미만 버림)를 신청인에게 지불해야 합니다.
3. 또한 피신청인은 세탁비 4,000원을 환급해야 합니다.

따라서, 이 사건 손해배상과 세탁비 환급은 신청인의 제품 반환과 피신청인의 금액 지불로 이루어집니다.
